### Installing reqs and downloading examples

In [1]:
# If package is not installed
! pip install git+https://github.com/salute-developers/GigaAM.git

In [2]:
# Downloading wavs for examples
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/example.wav
!wget https://n-ws-q0bez.s3pd12.sbercloud.ru/b-ws-q0bez-jpv/GigaAM/long_example.wav

## Speech Recognition

In [3]:
import os
import warnings
from typing import Dict

import gigaam

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [4]:
model = gigaam.load_model(
    "ctc",  # GigaAM-V2 CTC model
    fp16_encoder=True,  # to use fp16 encoder weights - GPU only
    use_flash=False,  # disable flash attention - colab does not support it
)
model.transcribe("example.wav")

'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'

In [5]:
model = gigaam.load_model(
    "rnnt",  # GigaAM-V2 RNNT model
    device="cpu",  # CPU-inference
)
model.transcribe("example.wav")

'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'

### Long-Form Speech Recognition

In [ ]:
!pip install gigaam[longform]

* For long-form inference:
  * generate [Hugging Face API token](https://huggingface.co/docs/hub/security-tokens)
  * accept the conditions to access [pyannote/voice-activity-detection](https://huggingface.co/pyannote/voice-activity-detection) files and content
  * accept the conditions to access [pyannote/segmentation](https://huggingface.co/pyannote/segmentation) files and content

In [ ]:
os.environ["HF_TOKEN"] = "<HF_TOKEN>"

In [ ]:
model = gigaam.load_model("ctc", use_flash=False)
recognition_result = model.transcribe_longform("long_example.wav")

for utterance in recognition_result:
    transcription = utterance["transcription"]
    start, end = utterance["boundaries"]
    print(f"[{gigaam.format_time(start)} - {gigaam.format_time(end)}]: {transcription}")

[00:00:00 - 00:16:83]: вечерня отошла давно но в кельях тихо и темно уже и сам игумен строгий свои молитвы прекратил и кости ветхие склонил перекрестясь на одр убогий кругом и сон и тишина но церкви дверь отворена
[00:17:10 - 00:32:61]: трепещет луч лампады и тускло озаряет он и темную живопись икон и позлощенные оклады и раздается в тишине то тяжкий вздох то шепот важный и мрачно дремлет в вышине старинный свод
[00:32:95 - 00:49:33]: глухой и влажный стоят за клиросом чернец и грешник неподвижны оба и шепот их как глаз из гроба и грешник бледен как мертвец монах несчастный полно перестань
[00:49:82 - 01:05:74]: ужасна исповедь злодея заплачена тобою дань тому кто в злобе пламенея лукаво грешника блюдет и к вечной гибели ведет смирись опомнись время время раскаянье покров
[01:05:97 - 01:10:90]: я разрешу тебя грехов сложи мучительное бремя


## Emotion recognition

In [6]:
model = gigaam.load_model("emo")
emotion2prob: Dict[str, int] = model.get_probs("example.wav")

print(", ".join([f"{emotion}: {prob:.3f}" for emotion, prob in emotion2prob.items()]))

angry: 0.000, sad: 0.002, neutral: 0.923, positive: 0.074


## GigaAM embeddings

In [7]:
# audio-only pretrained encoder
model = gigaam.load_model("ssl", use_flash=False)

emb, _ = model.embed_audio("example.wav")
print(emb)

tensor([[[-0.2815,  0.3648,  0.4501,  ..., -0.4732, -0.4029, -0.2401],
         [ 0.1610, -0.4995, -0.0576,  ..., -0.6241, -0.2316, -0.2052],
         [-1.1856, -1.0031, -0.6082,  ..., -0.5144, -0.3742, -0.2669],
         ...,
         [ 0.0188, -0.3748, -0.8955,  ...,  0.1732,  0.0576,  0.1311],
         [ 0.2698, -0.0659, -0.5008,  ..., -1.4415, -1.4807, -1.4491],
         [-1.5653, -1.6697, -1.2832,  ...,  0.5118,  0.4837,  0.0140]]],
       grad_fn=<TransposeBackward0>)


In [8]:
# you also can embed audio with CTC- or RNNT-finetuned encoder
model = gigaam.load_model("ctc", use_flash=False)

emb, _ = model.embed_audio("example.wav")
print(emb)

tensor([[[-1.2887, -0.4310, -0.5055,  ..., -0.2268, -0.6392, -0.7267],
         [-0.3969,  0.0309, -0.9771,  ...,  0.2250,  0.2793,  0.0538],
         [-0.8635, -1.1605, -0.9831,  ..., -1.1848, -1.4710, -1.4190],
         ...,
         [ 0.7540,  0.5017, -0.1576,  ...,  1.6958,  1.8045,  2.0394],
         [-0.0138, -0.7460, -1.1923,  ..., -0.8860, -0.8260, -0.8353],
         [-1.6170, -1.9980, -2.2678,  ...,  0.9713,  1.1955,  1.2046]]],
       grad_fn=<TransposeBackward0>)


## Export to ONNX

In [9]:
onnx_dir = "onnx"
model_type = "rnnt" # or "ctc"

model = gigaam.load_model(
    model_type,
    fp16_encoder=False,  # only fp32 tensors
    use_flash=False,  # disable flash attention
)
model.to_onnx(dir_path=onnx_dir)

Succesfully ported onnx v2_rnnt_encoder to onnx/v2_rnnt_encoder.onnx.
Succesfully ported onnx v2_rnnt_decoder to onnx/v2_rnnt_decoder.onnx.
Succesfully ported onnx v2_rnnt_joint to onnx/v2_rnnt_joint.onnx.


In [10]:
# Export emo model
onnx_dir = "onnx"
model_type = "emo"

model = gigaam.load_model(
    model_type,
    fp16_encoder=False,  # only fp32 tensors
    use_flash=False,  # disable flash attention
    )
model.to_onnx(dir_path=onnx_dir)

Succesfully ported onnx v1_emo to onnx/v1_emo.onnx.


### ONNX inference

In [11]:
from gigaam.onnx_utils import load_onnx_sessions, transcribe_sample

model_type = "rnnt"

sessions = load_onnx_sessions(onnx_dir, model_type)
transcribe_sample("example.wav", model_type, sessions)

'ничьих не требуя похвал счастлив уж я надеждой сладкой что дева с трепетом любви посмотрит может быть украдкой на песни грешные мои у лукоморья дуб зеленый'

In [12]:
# Emo model onnx inference
from gigaam.onnx_utils import load_onnx_sessions, recognise_emotion

model_type = "emo"
sessions = load_onnx_sessions(onnx_dir, model_type, model_version="v1")
recognise_emotion("example.wav", sessions)

{'angry': 7.712716615060344e-05,
 'sad': 0.0021968623623251915,
 'neutral': 0.9233159422874451,
 'positive': 0.07441005110740662}